In [40]:
import numpy as np
import pandas as pd
import requests
import time
import json
from pprint import pprint


In [127]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from cryptography.fernet import Fernet

# NYT Data Extract/Transform

In [6]:
# API KEY 
ses_api_key = "EH3LYl2x8c4wLClt63RJZGBrZboUE507"

# Best Sellers Lists
nyt_lists_qry = "https://api.nytimes.com/svc/books/v3/lists/names.json?api-key=" + ses_api_key 
nyt_lists_results = requests.get(nyt_lists_qry).json()
nyt_lists_results
nyt_lists_df = pd.DataFrame(nyt_lists_results['results'])

In [ ]:
nyt_lists_df.sort_values(['newest_published_date'], ascending = False)

In [10]:
# Hardcover NONFiction Best Sellers Extract
nyt_h_nfiction_qry = "https://api.nytimes.com/svc/books/v3/lists/current/hardcover-nonfiction.json?api-key=" + ses_api_key 
nyt_h_nfiction_results = requests.get(nyt_h_nfiction_qry).json()
nyt_h_nfiction_results
nyt_h_nfiction = nyt_h_nfiction_results['results']['books']

In [100]:
# Hardcover NONFiction Transform
nyt_h_nf_df = pd.DataFrame(nyt_h_nfiction)
nyt_h_nf_df = nyt_h_nf_df[['rank','weeks_on_list', 'primary_isbn13', 'publisher', 'book_review_link']]
nyt_h_nf_df = nyt_h_nf_df.rename(columns={'primary_isbn13':'book_id'})
nyt_h_nf_df['book_id']=nyt_h_nf_df['book_id'].astype(np.int64)



category = 'Hardcover Non-Fiction'
nyt_h_nf_df['category'] = category
date = pd.datetime.now()
nyt_h_nf_df['current_as_of_date'] = date

nyt_h_nf_df.head()

,rank,weeks_on_list,book_id,publisher,book_review_link,category,current_as_of_date
0,1,78,9780399590504,Random House,https://www.nytimes.com/2018/03/01/books/revie...,Hardcover Non-Fiction,2019-08-23 09:51:01.753481
1,2,1,9780525509288,One World,,Hardcover Non-Fiction,2019-08-23 09:51:01.753481
2,3,40,9781524763138,Crown,https://www.nytimes.com/2018/12/06/books/revie...,Hardcover Non-Fiction,2019-08-23 09:51:01.753481
3,4,6,9781451642292,Avid Reader,https://www.nytimes.com/2019/06/28/books/revie...,Hardcover Non-Fiction,2019-08-23 09:51:01.753481
4,5,15,9781501168680,Simon & Schuster,https://www.nytimes.com/2019/05/13/books/revie...,Hardcover Non-Fiction,2019-08-23 09:51:01.753481


In [15]:
# Paperback NONFiction Best Sellers
nyt_p_nfiction_qry = "https://api.nytimes.com/svc/books/v3/lists/current/paperback-nonfiction.json?api-key=" + ses_api_key 
nyt_p_nfiction_results = requests.get(nyt_p_nfiction_qry).json()
nyt_p_nfiction_results
nyt_p_nfiction = nyt_p_nfiction_results['results']['books']

In [99]:
# Paperback NONFiction Transform
nyt_p_nf_df = pd.DataFrame(nyt_p_nfiction)
nyt_p_nf_df = nyt_p_nf_df[['rank','weeks_on_list', 'primary_isbn13', 'publisher', 'book_review_link']]
nyt_p_nf_df = nyt_p_nf_df.rename(columns={'primary_isbn13':'book_id'})
nyt_p_nf_df['book_id']=nyt_p_nf_df['book_id'].astype(np.int64)

category = 'Paperback Non-Fiction'
nyt_p_nf_df['category'] = category
date = pd.datetime.now()
nyt_p_nf_df['current_as_of_date'] = date

nyt_p_nf_df.head()

,rank,weeks_on_list,book_id,publisher,book_review_link,category,current_as_of_date
0,1,27,9780399588198,Spiegel & Grau,,Paperback Non-Fiction,2019-08-23 09:50:54.819094
1,2,66,9780062316110,Harper Perennial,,Paperback Non-Fiction,2019-08-23 09:50:54.819094
2,3,58,9780807047415,Beacon Press,,Paperback Non-Fiction,2019-08-23 09:50:54.819094
3,4,166,9780812984965,Spiegel & Grau,,Paperback Non-Fiction,2019-08-23 09:50:54.819094
4,5,43,9780143127741,Penguin,,Paperback Non-Fiction,2019-08-23 09:50:54.819094


In [22]:
# Hardcover Fiction Best Sellers
nyt_h_fiction_qry = "https://api.nytimes.com/svc/books/v3/lists/current/hardcover-fiction.json?api-key=" + ses_api_key 
nyt_h_fiction_results = requests.get(nyt_h_fiction_qry).json()
nyt_h_fiction_results
nyt_h_fiction = nyt_h_fiction_results['results']['books']

In [101]:
# Hardcover Fiction Transform
nyt_h_f_df = pd.DataFrame(nyt_h_fiction)
nyt_h_f_df = nyt_h_f_df[['rank','weeks_on_list', 'primary_isbn13', 'publisher', 'book_review_link']]
nyt_h_f_df = nyt_h_f_df.rename(columns={'primary_isbn13':'book_id'})
nyt_h_f_df['book_id']=nyt_h_f_df['book_id'].astype(np.int64)

category = 'Hardcover Fiction'
nyt_h_f_df['category'] = category
date = pd.datetime.now()
nyt_h_f_df['current_as_of_date'] = date

nyt_h_f_df.head()

,rank,weeks_on_list,book_id,publisher,book_review_link,category,current_as_of_date
0,1,50,9780735219090,Putnam,,Hardcover Fiction,2019-08-23 09:51:33.678612
1,2,2,9780316527583,"Little, Brown",,Hardcover Fiction,2019-08-23 09:51:33.678612
2,3,1,9781250051059,Minotaur,,Hardcover Fiction,2019-08-23 09:51:33.678612
3,4,4,9781538750568,Grand Central,,Hardcover Fiction,2019-08-23 09:51:33.678612
4,5,5,9780385537070,Doubleday,https://www.nytimes.com/2019/07/11/books/revie...,Hardcover Fiction,2019-08-23 09:51:33.678612


In [29]:
# Trade Fiction Paperback Best Sellers
nyt_tfp_qry = "https://api.nytimes.com/svc/books/v3/lists/current/trade-fiction-paperback.json?api-key=" + ses_api_key 
nyt_tfp_results = requests.get(nyt_tfp_qry).json()
nyt_tfp_results
nyt_tfp = nyt_tfp_results['results']['books']

In [102]:
# Trade Fiction Paperback Transform
nyt_tfp_df = pd.DataFrame(nyt_tfp)
nyt_tfp_df = nyt_tfp_df[['rank','weeks_on_list', 'primary_isbn13', 'publisher', 'book_review_link']]
nyt_tfp_df = nyt_tfp_df.rename(columns={'primary_isbn13':'book_id'})
nyt_tfp_df['book_id']=nyt_tfp_df['book_id'].astype(np.int64)

category = 'Trade Fiction Paperback'
nyt_tfp_df['category'] = category
date = pd.datetime.now()
nyt_tfp_df['current_as_of_date'] = date

nyt_tfp_df.head()

,rank,weeks_on_list,book_id,publisher,book_review_link,category,current_as_of_date
0,1,164,9780062364913,Harper Perennial,,Trade Fiction Paperback,2019-08-23 09:51:53.995755
1,2,50,9780062797155,Harper,https://www.nytimes.com/2018/11/16/books/revie...,Trade Fiction Paperback,2019-08-23 09:51:53.995755
2,3,15,9780735224315,Penguin,https://www.nytimes.com/2017/09/25/books/revie...,Trade Fiction Paperback,2019-08-23 09:51:53.995755
3,4,2,9781400033416,Vintage,https://www.nytimes.com/1987/09/02/books/books...,Trade Fiction Paperback,2019-08-23 09:51:53.995755
4,5,13,9780425284704,Ballantine,,Trade Fiction Paperback,2019-08-23 09:51:53.995755


In [31]:
# YA Hardcover Best Sellers
nyt_ya_qry = "https://api.nytimes.com/svc/books/v3/lists/current/young-adult-hardcover.json?api-key=" + ses_api_key 
nyt_ya_results = requests.get(nyt_ya_qry).json()
nyt_ya_results
nyt_ya = nyt_ya_results['results']['books']

In [103]:
# YA Hardcover Transform
nyt_ya_df = pd.DataFrame(nyt_ya)
nyt_ya_df = nyt_ya_df[['rank','weeks_on_list', 'primary_isbn13', 'publisher', 'book_review_link']]
nyt_ya_df = nyt_ya_df.rename(columns={'primary_isbn13':'book_id'})
nyt_ya_df['book_id']=nyt_ya_df['book_id'].astype(np.int64)

category = 'YA Hardcover'
nyt_ya_df['category'] = category
date = pd.datetime.now()
nyt_ya_df['current_as_of_date'] = date

nyt_ya_df.head()

,rank,weeks_on_list,book_id,publisher,book_review_link,category,current_as_of_date
0,1,129,9780062498533,Balzer + Bray,,YA Hardcover,2019-08-23 09:52:12.089104
1,2,37,9781534451568,Simon & Schuster,,YA Hardcover,2019-08-23 09:52:12.089104
2,3,17,9780062662804,HarperTeen,,YA Hardcover,2019-08-23 09:52:12.089104
3,4,76,9781250170972,Holt,,YA Hardcover,2019-08-23 09:52:12.089104
4,5,87,9781524714680,Delacorte,,YA Hardcover,2019-08-23 09:52:12.089104


In [130]:
nyt_books = pd.concat([nyt_h_nf_df, nyt_p_nf_df, nyt_h_f_df, nyt_tfp_df, nyt_ya_df]).reset_index()
del nyt_books['index']
nyt_books.index.name = 'reviews_id'
nyt_books.head()

,rank,weeks_on_list,book_id,publisher,book_review_link,category,current_as_of_date
reviews_id,,,,,,,
0,1,78,9780399590504,Random House,https://www.nytimes.com/2018/03/01/books/revie...,Hardcover Non-Fiction,2019-08-23 09:51:01.753481
1,2,1,9780525509288,One World,,Hardcover Non-Fiction,2019-08-23 09:51:01.753481
2,3,40,9781524763138,Crown,https://www.nytimes.com/2018/12/06/books/revie...,Hardcover Non-Fiction,2019-08-23 09:51:01.753481
3,4,6,9781451642292,Avid Reader,https://www.nytimes.com/2019/06/28/books/revie...,Hardcover Non-Fiction,2019-08-23 09:51:01.753481
4,5,15,9781501168680,Simon & Schuster,https://www.nytimes.com/2019/05/13/books/revie...,Hardcover Non-Fiction,2019-08-23 09:51:01.753481


# Goodreads Data Extract/Transform

# PenguinRH Data Extract/Transform

# Loading Data to Postgres

NYT DATA LOAD

In [129]:
#NYT DATA LOAD
# Dependent on the Goodreads/BooksDB section 

# Deal with password needed for sql conn
key = b'R_frnvbIOy25RMAXZtlk5yOn9x6m71YhQnfkpr2QM5w='
cipher_suite = Fernet(key)

with open('postgres.bin', 'rb') as file_object:
    for line in file_object:
        encryptedpwd = line
        
uncipher_text = (cipher_suite.decrypt(encryptedpwd))
pword = bytes(uncipher_text).decode("utf-8")

engine = create_engine('postgresql://postgres:' + pword + '@localhost:5432/ETL')
connection = engine.connect()
engine.table_names()

common_books = booksDB.merge(nyt_books,on=['book_id'])
common_books = common_books[['rank','weeks_on_list', 'book_id', 'publisher', 'book_review_link', 'category', 'current_as_of_date']]
common_books.index.name = 'reviews_id'

common_books.to_sql(name='reviews', con=engine, if_exists='append', index=True)

['county', 'premise', 'books', 'reviews']